In [1]:
!git clone https://github.com/wandb/tiny-ml.git

Cloning into 'tiny-ml'...
remote: Enumerating objects: 11630, done.
remote: Counting objects: 100% (317/317), done.
remote: Compressing objects: 100% (154/154), done.
remote: Total 11630 (delta 160), reused 314 (delta 158), pack-reused 11313
Receiving objects: 100% (11630/11630), 689.27 MiB | 13.22 MiB/s, done.
Resolving deltas: 100% (2365/2365), done.


In [1]:
import tensorflow as tf
import wandb
import audiomentations
from pathlib import Path

from scipy.io.wavfile import read
import pandas as pd
import plotly.express as px
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import sounddevice as sd
import time
import struct
import wave
from pvrecorder import PvRecorder
import IPython
import os

# some utilities that weve made in 

import recorder
from utils.data_processing import augmenter,log_wandb_artifact,plot_spectrogram,segment,read_wav,Arm_spect


In [2]:
os.environ["WANDB_SILENT"] = "false"

In [3]:
from traitlets.config.manager import BaseJSONConfigManager
from pathlib import Path
path = Path.home() / ".jupyter" / "nbconfig"
cm = BaseJSONConfigManager(config_dir=str(path))
cm.update(
    "rise",
    {
        "theme": "sky",
        "transition": "zoom",
        "start_slideshow_at": "selected",
        'scroll':'false'
     }
)

{'theme': 'sky',
 'transition': 'zoom',
 'start_slideshow_at': 'selected',
 'scroll': 'false'}

![](slides/tiny_ml_cover.svg)

![](slides/ml_morphology.svg)

![](slides/mlops_graph.svg)

![](slides/edge_devices.svg)

<!DOCTYPE html>
<html>
<table style="height:1000px;width:100%">
 <tr>
    <td> <img src="./slides/spark_micro_mod_pico_1.jpg" alt="Drawing" style="height: 525px;width:800px;" align="right"/> </td>
    <td> <img src="./slides/micro_mod_pico_2.jpg" alt="Drawing" style="height: 850px; width:900px" align="left"/> </td>
    </tr>
</table>

In [4]:
tqdm_args = dict(leave=True,position=0)
tqdm_colours = (f'#{r:02x}{g:02x}{b:02x}'for r,g,b in [(int(255*s), 3, int(255*s)) for s in np.linspace(1,0,100)])
run = wandb.init(entity='tiny-ml',project = 'wake_word_detection', group='Data')

wandb: Currently logged in as: frds (tiny-ml). Use `wandb login --relogin` to force relogin


In [5]:
recorder = PvRecorder(device_index=0, frame_length=512)
sound = []
recorder.start()
print('recording')
t_0 = time.time()
while time.time()-t_0<4:
    frame = recorder.read()
    sound.extend(frame)
recorder.stop()

recording


In [6]:
data = np.array(sound).astype(np.int16)
t_step = np.arange(0, len(data))
fig = px.line(x=t_step, y=data, title='a sound wave')
run.log({'sound_wave': fig})

### Showing a Run in Notebook

In [7]:
run

In [8]:
path = './data/yes/yes_record 102.wav'
run.log({'test_sound':wandb.Audio(path)})
run

# The basis of our signal processing is using Fast Fourier Transform (FFT)
> - [here](https://www.youtube.com/watch?v=spUNpyF58BY) is an amazing video on how FFT works. 
> - quickly go through getting from FFT to a Spectrogram. 
>  ![](./slides/200px-Fourier_in_his_coat_of_prefect.jpg)

In [9]:
fft = np.fft.fft(data).astype(np.int16)
t_steps = np.arange(0,len(fft))
fft = np.stack([fft,t_step]).T
df = pd.DataFrame(fft, columns = ['Faurrier_Value','Time_Step'])
fig = px.line(df,x='Time_Step', y='Faurrier_Value', title='Fourrier Transform')
run.log({'Fouriere Transform':fig})

/var/folders/q8/9dhs6y4d21s87vxtnwqflq5m0000gn/T/ipykernel_40417/660930182.py:1: ComplexWarning:

Casting complex values to real discards the imaginary part

Exception in thread SystemMonitor:
Traceback (most recent call last):
  File "/Users/fridadesigley/.pyenv/versions/3.10.6/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/Users/fridadesigley/.pyenv/versions/3.10.6/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/fridadesigley/.pyenv/versions/3.10.6/envs/tiny-ml/lib/python3.10/site-packages/wandb/sdk/internal/system/system_monitor.py", line 118, in _start
    asset.start()
  File "/Users/fridadesigley/.pyenv/versions/3.10.6/envs/tiny-ml/lib/python3.10/site-packages/wandb/sdk/internal/system/assets/cpu.py", line 166, in start
    self.metrics_monitor.start()
  File "/Users/fridadesigley/.pyenv/versions/3.10.6/envs/tiny-ml/lib/python3.10/site-packages/wandb/sdk/internal/system/assets/interf

In [10]:
run

## FFT can also be inverted, we can get sound back

In [11]:
fft = np.fft.fft(data)
roll = np.roll(fft,200)
ifft = np.fft.ifft(roll).astype(np.int16)
t_step = np.arange(len(ifft))
ifft = np.stack([ifft,t_step]).T
df = pd.DataFrame(ifft, columns = ['Frequecy_Value','Time_Step'])
fig = px.line(df,x='Time_Step', 
              y='Frequecy_Value', 
              title='Inverse Fourrier Transform to Get sound Back')
run.log({'Inverset Fouriere':fig})

/var/folders/q8/9dhs6y4d21s87vxtnwqflq5m0000gn/T/ipykernel_40417/1994990191.py:3: ComplexWarning:

Casting complex values to real discards the imaginary part



In [12]:
run

## Down Sample using FFT as an example of Signal Processing

In [13]:
# and examper of downsampling using FFT
roll = np.roll(fft,15)
ifft_ten = np.fft.ifft(roll)
ifft_ten = ((2**(16-4)) * ifft_ten/ifft_ten.max()).astype(np.int16)
ifft_ten = np.stack([ifft_ten,t_step]).T
df = pd.DataFrame(ifft_ten, columns = ['Frequecy_Value','Time_Step'])
fig = px.line(df,x='Time_Step', y='Frequecy_Value', title='Dowsampling using Fast Fourrier Transform (FFT)')
run.log({'Down_Sampled_Sound':fig})

/var/folders/q8/9dhs6y4d21s87vxtnwqflq5m0000gn/T/ipykernel_40417/3482508054.py:4: ComplexWarning:

Casting complex values to real discards the imaginary part



In [14]:
run

In [15]:
run.finish()
run = wandb.init(id=run.id,entity='tiny-ml',project = 'wake_word_detection', group='Data',
                resume='must')

### ARM spectrogram using FFT 🦾


In [16]:
category = 'yes'
idx = 10
sound = read_wav(f'./data/{category}/{category}_record {idx}.wav')
sound = sound.astype(np.float32, order='F') / 32768.0
get_arm_spectrogram = Arm_spect().get_arm_spectrogram
get_arm_spectrogram(sound)

array([[0.5     , 0.3125  , 0.125   , ..., 0.      , 0.      , 0.      ],
       [0.671875, 0.375   , 0.0625  , ..., 0.015625, 0.      , 0.      ],
       [0.78125 , 0.390625, 0.03125 , ..., 0.      , 0.      , 0.      ],
       ...,
       [1.109375, 0.53125 , 0.125   , ..., 0.015625, 0.      , 0.      ],
       [0.96875 , 0.375   , 0.03125 , ..., 0.015625, 0.      , 0.      ],
       [0.921875, 0.34375 , 0.0625  , ..., 0.      , 0.      , 0.      ]],
      dtype=float32)

### Showing output of custom function optimized for arm processor

In [17]:
category = 'yes'
sound = read_wav(f'./data/{category}/yes_record 119.wav')
print(sound)
sound = sound.astype(np.float32, order='F') / 32768.0
# what we are using 
arm_spct = get_arm_spectrogram(sound)
# what were not using 
tf_spect = tf.signal.stft(sound, frame_length=512, frame_step=128).numpy().astype(np.float32)
fig = px.imshow(np.array(arm_spct))
run.log({f'spectrogram_{category}':fig})

[   0    0    0 ... -591 -643 -631]


/var/folders/q8/9dhs6y4d21s87vxtnwqflq5m0000gn/T/ipykernel_40417/3992365391.py:8: ComplexWarning:

Casting complex values to real discards the imaginary part



In [18]:
run 

# Adding spectrogram plot to wandb table 🌌

# logging some media outside of a table to workspace
This is for:
- cheching what our network see in signal domain;
- to spot patterns;
- gain intuitions about process and raw signal data.

In [19]:
sample_rate = 16000
category = 'yes'
n_samples = 10
for idx in tqdm(range(1,n_samples), colour=next(tqdm_colours)):
    sound = read_wav(f'./data/{category}/{category}_record {idx}.wav')
    sound = sound.astype(np.float32, order='F') / 32768.0
    arm_spect = get_arm_spectrogram(sound)
    img = plot_spectrogram(arm_spect,sample_rate=sample_rate)
    img = wandb.Image(img)
    run.log({f'spectrogram_image{category}':img})

100%|█████████████████████████████████████████████| 9/9 [00:02<00:00,  3.02it/s]


In [20]:
run

In [21]:
#  !mkdir datasets
#  !wget 'https://github.com/karoldvl/ESC-50/archive/master.zip' -P ~/pico/audio-classifier/datasets
#  !unzip -q ~/datasets/master.zip -d ./datasets/

# This Part covers pre training our model on ESC 50 dataset

### Reading our pre-traning data to pandas

In [22]:
run = wandb.init(id=run.id,
                 entity='tiny-ml',
                 project = 'wake_word_detection', 
                 group='Data',
                 resume='must')
esc50_csv = './datasets/ESC-50-master/meta/esc50.csv'
df = pd.read_csv(esc50_csv)
initial_data_table = wandb.Table(data=df, columns=list(df.columns))
run.log({'initial_table':initial_data_table})

# building a wandb Artifact for the generic dataset for pre training

In [23]:
from pathlib import Path

artifact = wandb.Artifact(type='sound_data',
                          name='ESC-50-master')
artifact.add_dir('./datasets/')
run.log_artifact(artifact)

wandb: Adding directory to artifact (./datasets)... Done. 1.8s


### Here we are training on 1 second sound clips and our ESC 50 Dataset is 4 seconds per class

### testing our function 

In [24]:
test_file = 'datasets/ESC-50-master/audio/1-100032-A-0.wav'
data = segment(fid=test_file,chunk=1)

### reading 1 second clips writing to a new files

In [25]:
in_paranet = Path('./datasets/ESC-50-master/audio/')
out_data_dir = Path('ESC-50')
out_data_dir.mkdir(exist_ok=True)
all_out = [ ]
for idx, row in tqdm(df.iterrows()):
    out_dir = out_data_dir/row.category
    out_dir.mkdir(exist_ok=True)
    data, vals = segment(fid = str(in_paranet/row.filename),
       chunk=1)
    chans, samp_width , rate = vals
    out_fids = [ ]
    for idx,sound in enumerate(data):
        out_fid = out_dir/f'{idx}_{row.filename}'
        out_fids.append(out_fid)
        with wave.open(str(out_fid), 'w') as outfile:
            outfile.setnchannels(chans)
            outfile.setsampwidth(samp_width)
            outfile.setframerate(rate)
            outfile.setnframes(int(len(sound) /  samp_width))
            outfile.writeframes(sound)
    all_out.append(out_fids)


12000it [00:10, 1101.45it/s]


# Building an Artifact of 1 second sound clips broken down into class/category name
This:
> - creates a type which is here a parent directory
> - names an artfact for each class
> - creates a list of artfacts which are then itterated through to add a dirctory

#### Our artifacts are going to follow this pattern

In [26]:
import shutil
run = wandb.init(id=run.id,entity='tiny-ml',project = 'wake_word_detection', group='Data',
                resume='must')

esc50_csv = Path('./datasets/ESC-50-master/meta/esc50.csv')
path = Path('ESC-50/')
meta_data = path/'meta_data'
meta_data.mkdir(exist_ok=True)
shutil.copyfile(esc50_csv,meta_data/esc50_csv.name)
esc_artifacts = [pth for pth in path.iterdir()]
for path in tqdm(esc_artifacts[-15:],**tqdm_args,colour=next(tqdm_colours)):
    log_wandb_artifact(run=run,path=path)
    
run.finish()


100%|███████████████████████████████████████████| 15/15 [00:04<00:00,  3.10it/s]


wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.


### adding our one second sound file paths to our data frame

In [27]:
run = wandb.init(id=run.id,entity='tiny-ml',project = 'wake_word_detection', group='Data',
                resume='must')
df = pd.read_csv(esc50_csv)
esc50_csv = meta_data/esc50_csv.name
sec_files = np.array(all_out).astype(str)
for files in range(sec_files.shape[-1]):
    df[f'{files}_{files+1}_sec']=sec_files[...,files].astype(str)
df.to_csv(esc50_csv)
log_wandb_artifact(run=run,path=esc50_csv)

wandb: Adding directory to artifact (./ESC-50/meta_data)... Done. 0.0s


In [28]:
run

In [29]:
meta_data_table = wandb.Table(data=df,columns=list(df.columns))
run.log({'meta_data_without_media':meta_data_table})


# Data table for sound Wandb table with 

> 1. ouriginal sound string
> 2. playable sound file
> 3. 4 * 1 second clips files
> 4  4 * 1 second clips files

# Sound and Spectrogram Table



## Adding Sound and Spectrograms to WandB Table  🔉 🪄 🐝

In [30]:
run = wandb.init(id=run.id,entity='tiny-ml',project = 'wake_word_detection', group='Data',
                resume='must')
n_examples = 2
n_samples = 10
s_df = df.sample(n=n_samples, random_state=2)
columns = list(df.columns)
table_with_media = wandb.Table(data=s_df,columns=columns)
for colmn in df.columns[-n_examples:]:
    # create wandb sound objects
    sounds = [wandb.Audio(fid) for fid in tqdm(s_df[colmn].values,**tqdm_args,desc=colmn)]
    # add these as a column to our wandb table
    table_with_media.add_column(name=f'sound_{colmn}',data=sounds)

5_6_sec: 100%|████████████████████████████████| 10/10 [00:00<00:00, 2604.83it/s]


### Adding Spectrograms to wandb Table

In [31]:
for col in df.columns[-n_examples:]:
    # read wavs 
    wavs = map(read_wav,tqdm(s_df[col].values,**tqdm_args,desc=col))
    # generate spectrograms
    spects = map(get_arm_spectrogram,wavs)
    sample_rates = iter(np.full((1,len(df)),1600)[0])
    # get numpy arrays of save .jpg files
    ims = list(map(plot_spectrogram,spects,sample_rates))
    # create a list of wandb images
    spects = list(map(wandb.Image,tqdm(ims)))
    # add these to a table 
    table_with_media.add_column(name=f'image_{col}',data=spects)
run.log({'sound_spectrogram_table':table_with_media})

100%|███████████████████████████████████████████| 10/10 [00:07<00:00,  1.33it/s]


## Stack our pandas df by 1 second Files 🧱

In [32]:
rp_df = pd.DataFrame(np.repeat(df.values, 6, axis=0))
rp_df.columns=df.columns
a,b,c,d,e,f = np.array([df[col].values for col in df.columns[-6:]])
all_files =np.vstack((a,b,c,d,e,f)).reshape((-1,),order='F')
rp_df = rp_df[df.columns[:-6]]
rp_df['all_files']=all_files
df = rp_df
df.to_csv(esc50_csv)
log_wandb_artifact(run,esc50_csv)

wandb: Adding directory to artifact (./ESC-50/meta_data)... Done. 0.0s


### Define Test,Train, Val using pandas & add sets column to our dataset

In [33]:
sets = { }
train_val = df.sample(frac=0.9,random_state=42)
sets['test']=df.drop(train_val.index)
sets['train']=train_val.sample(frac=0.9,random_state=42)
sets['val']=train_val.drop(sets['train'].index)
for name,subset in sets.items():
        subset = subset.reset_index(drop=True)
        subset['set']=[name for va in range(len(subset))]
        sets[name]=subset
df.groupby("target", group_keys=False).apply(lambda x: x)
df = pd.concat(sets.values()).reset_index(drop=True)
df.to_csv(esc50_csv)
log_wandb_artifact(run,esc50_csv)

wandb: Adding directory to artifact (./ESC-50/meta_data)... Done. 0.0s


In [34]:
df

,Unnamed: 0,filename,fold,target,category,esc10,src_file,take,all_files,set
0,3,1-103995-A-30.wav,1,30,door_wood_knock,False,103995,A,ESC-50/door_wood_knock/3_1-103995-A-30.wav,test
1,4,1-103999-A-30.wav,1,30,door_wood_knock,False,103999,A,ESC-50/door_wood_knock/4_1-103999-A-30.wav,test
2,9,1-115546-A-48.wav,1,48,fireworks,False,115546,A,ESC-50/fireworks/1_1-115546-A-48.wav,test
3,10,1-115920-A-22.wav,1,22,clapping,False,115920,A,ESC-50/clapping/0_1-115920-A-22.wav,test
4,11,1-115920-B-22.wav,1,22,clapping,False,115920,B,ESC-50/clapping/0_1-115920-B-22.wav,test
...,...,...,...,...,...,...,...,...,...,...
71995,7488,4-204115-A-39.wav,4,39,glass_breaking,False,204115,A,ESC-50/glass_breaking/4_4-204115-A-39.wav,val
71996,9642,2-118072-A-0.wav,2,0,dog,True,118072,A,ESC-50/dog/5_2-118072-A-0.wav,val
71997,6280,4-189836-A-22.wav,4,22,clapping,False,189836,A,ESC-50/clapping/4_4-189836-A-22.wav,val
71998,1177,5-252248-A-34.wav,5,34,can_opening,False,252248,A,ESC-50/can_opening/3_5-252248-A-34.wav,val


# remove silence by randomly sampling from non silent

In [35]:
run = wandb.init(id=run.id,entity='tiny-ml',project = 'wake_word_detection', group='Data',
                resume='must')
is_sound = [not np.std(read_wav(x)) < 0.1 for x in df.all_files.values]
df['is_sound']=is_sound
df.to_csv(esc50_csv)
log_wandb_artifact(run,esc50_csv)

wandb: Adding directory to artifact (./ESC-50/meta_data)... Done. 0.0s


In [36]:
silence = df[df.is_sound==False]
final_meta_table = wandb.Table(data=silence,columns=list(df.columns))
wandb_audio = list(map(wandb.Audio,silence.all_files.values))
final_meta_table.add_column(name='audio',data=wandb_audio)
run.log({'silence_tabel':final_meta_table})

In [37]:
run

# Building a wandb Artifact for our pre processed data (numpy array spectrograms):

In [38]:
run = wandb.init(id=run.id,entity='tiny-ml',project = 'wake_word_detection', group='Data',
                resume='must')
artifact = wandb.Artifact(type='pre_processed_sound_data',name='npz-esc-50-files')
path = Path('./processed/')
# create references for our large pre processed npz files
for fid_path in path.iterdir():
    print(f'file://{fid_path.resolve()}')  
    artifact.add_reference(f'file://{fid_path.resolve()}')
run.log_artifact(artifact)

wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.
wandb: Network error (TransientError), entering retry loop.


file:///Users/fridadesigley/pico/tiny-ml/processed/train.npz
file:///Users/fridadesigley/pico/tiny-ml/processed/train_data.npz
file:///Users/fridadesigley/pico/tiny-ml/processed/test.npz
file:///Users/fridadesigley/pico/tiny-ml/processed/val.npz
file:///Users/fridadesigley/pico/tiny-ml/processed/train_aug.npz


In [40]:
for pth in df[df.is_sound==False].all_files.values:
    path = Path(pth)
    try:
        path.unlink()
    except FileNotFoundError:
for path in tqdm(esc_artifacts,**tqdm_args,colour=next(tqdm_colours)):
    log_wandb_artifact(run=run,path=path)

alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy del

alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy del

alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy deleted
alredy del

100%|███████████████████████████████████████████| 51/51 [00:14<00:00,  3.59it/s]


# Save 3D arrays of Spectrograms using numpy

In [89]:
def save_npz(x,y,name,augmenter=None):
    wavs = list(map(read_wav,tqdm(x,f'creating {name}')))
    if augmenter:
        samples = np.full((1,len(y)), 16000)[0]
        wavs = list(map(augmenter,wavs,tqdm(samples,f'augmenting {name}')))
        x_data = np.array(list(map(get_arm_spectrogram,tqdm(wavs,f'creating {name} augspects'))))
    else:
        x_data = np.array(list(map(get_arm_spectrogram,tqdm(wavs,f'creating {name} spects'))))
    for array in x_data:
        assert array.shape==(682, 257)
    np.savez(f'{name}.npz',x_data=x_data,y_data=y)

## Loop over files and save as npz, this is our pre-processed training data.

In [41]:
contains_sound = df[df.is_sound==True]
subset =  contains_sound[contains_sound.set=='train']


In [ ]:
pre_process=True
contains_sound = df[df.is_sound==True]
if pre_process:
    for name in df.set.unique():
        subset =  contains_sound[contains_sound.set==name]
        x_data = subset['all_files'].values
        y_data = subset['target'].values
        save_npz(x_data,y_data,f'processed/{name}')
        if name=='train':
            print('augmenting')
            save_npz(x_data[:len(x_data)//5],y_data[:len(y_data)//5],f'processed/{name}_aug',augmenter=augmenter)

creating processed/train spects: 100%|███| 55257/55257 [07:23<00:00, 124.57it/s]


# Building a wandb Artifact for our pre processed data (numpy array spectrograms):

In [46]:
artifact = wandb.Artifact(type='pre_processed_sound_data',name='npz-esc-50-files')
path = Path('./processed/')
# create references for our large pre processed npz files
for fid_path in path.iterdir():
    print(f'file://{fid_path.resolve()}')  
    artifact.add_reference(f'file://{fid_path.resolve()}')
run.log_artifact(artifact)

file:///Users/fridadesigley/pico/tiny-ml/processed/train.npz
file:///Users/fridadesigley/pico/tiny-ml/processed/train_data.npz
file:///Users/fridadesigley/pico/tiny-ml/processed/test.npz
file:///Users/fridadesigley/pico/tiny-ml/processed/val.npz
file:///Users/fridadesigley/pico/tiny-ml/processed/train_aug.npz


### To finish our data run

In [47]:
run.finish()
run

Exception in thread SockSrvRdThr:
Traceback (most recent call last):
  File "/Users/fridadesigley/.pyenv/versions/3.10.6/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/Users/fridadesigley/.pyenv/versions/3.10.6/envs/tiny-ml/lib/python3.10/site-packages/wandb/sdk/service/server_sock.py", line 112, in run
    shandler(sreq)
  File "/Users/fridadesigley/.pyenv/versions/3.10.6/envs/tiny-ml/lib/python3.10/site-packages/wandb/sdk/service/server_sock.py", line 165, in server_record_communicate
    iface = self._mux.get_stream(stream_id).interface
  File "/Users/fridadesigley/.pyenv/versions/3.10.6/envs/tiny-ml/lib/python3.10/site-packages/wandb/sdk/service/streams.py", line 206, in get_stream
    stream = self._streams[stream_id]
KeyError: 'w0790zli'


# Part 2. Training

### Training load training dataset which has already been pre processed

In [130]:
with np.load('processed/train.npz',allow_pickle=True) as data:
    train_x = data['x_data'].astype(np.float32)
    train_y = data['y_data'].astype(np.uint8)
with np.load('processed/train_aug.npz',allow_pickle=True) as data:
    aug_x = data['x_data'].astype(np.float32)
    aug_y = data['y_data'].astype(np.uint8)
with np.load('processed/test.npz',allow_pickle=True) as data:
    test_x = data['x_data'].astype(np.float32)
    test_y = data['y_data'].astype(np.uint8)
with np.load('processed/val.npz',allow_pickle=True) as data:
    val_x = data['x_data'].astype(np.float32)
    val_y = data['y_data'].astype(np.uint8)
    


In [74]:
type(train_x)

numpy.ndarray

## create TensoFlow Dataset from numpy arrays of spectrograms and target variable

In [134]:
train_dataset = tf.data.Dataset.from_tensor_slices((tf.cast(train_x, tf.float32), train_y))
train_aug_dataset = tf.data.Dataset.from_tensor_slices((tf.cast(aug_x, tf.float32), aug_y))
train_dataset = train_dataset.concatenate(train_aug_dataset)
val_dataset = tf.data.Dataset.from_tensor_slices((tf.cast(val_x, tf.float32),val_y))
test_dataset = tf.data.Dataset.from_tensor_slices((tf.cast(test_x,tf.float32),test_y))

train_ds = train_dataset.cache().shuffle(10000, seed=42).batch(32).prefetch(tf.data.AUTOTUNE)
val_ds = val_dataset.cache().batch(32).prefetch(tf.data.AUTOTUNE)
test_ds = test_dataset.cache().batch(32).prefetch(tf.data.AUTOTUNE)

In [136]:
x, y = next(iter(train_ds))
x[0],y[0]

2023-01-29 21:37:29.267569: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


(<tf.Tensor: shape=(682, 257), dtype=float32, numpy=
 array([[61.296875, 33.65625 ,  3.59375 , ...,  1.0625  ,  1.546875,
          0.34375 ],
        [63.8125  , 37.5     ,  7.84375 , ...,  1.046875,  1.484375,
          0.734375],
        [59.984375, 36.3125  , 14.375   , ...,  1.59375 ,  1.53125 ,
          0.390625],
        ...,
        [39.84375 , 38.78125 , 15.5625  , ...,  0.796875,  0.765625,
          0.46875 ],
        [25.90625 , 33.734375, 26.21875 , ...,  1.09375 ,  1.375   ,
          1.109375],
        [14.640625, 18.078125, 42.578125, ...,  1.453125,  1.765625,
          1.484375]], dtype=float32)>,
 <tf.Tensor: shape=(), dtype=uint8, numpy=43>)

## Test that this dataset is loading okay

In [137]:
x,y = next(iter(train_dataset.take(1)))
print(f' target = {y}, \n spectrogram = \n {x}')
input_shape = tf.expand_dims(x, axis=-1).shape
print(input_shape)

 target = 10, 
 spectrogram = 
 [[14.6875   10.984375  8.125    ...  2.484375  2.765625  4.21875 ]
 [ 7.09375  10.1875   10.484375 ...  2.140625  1.921875  4.453125]
 [ 0.609375  9.265625 14.96875  ...  1.671875  1.859375  4.484375]
 ...
 [ 2.265625  4.453125  3.75     ...  5.6875    4.59375   0.40625 ]
 [ 1.1875    5.765625  6.4375   ...  5.3125    4.03125   2.484375]
 [ 4.453125  9.6875    7.171875 ...  3.203125  3.375     2.734375]]
(682, 257, 1)


In [138]:
print(x)

tf.Tensor(
[[14.6875   10.984375  8.125    ...  2.484375  2.765625  4.21875 ]
 [ 7.09375  10.1875   10.484375 ...  2.140625  1.921875  4.453125]
 [ 0.609375  9.265625 14.96875  ...  1.671875  1.859375  4.484375]
 ...
 [ 2.265625  4.453125  3.75     ...  5.6875    4.59375   0.40625 ]
 [ 1.1875    5.765625  6.4375   ...  5.3125    4.03125   2.484375]
 [ 4.453125  9.6875    7.171875 ...  3.203125  3.375     2.734375]], shape=(682, 257), dtype=float32)


In [139]:
norm_layer = tf.keras.layers.experimental.preprocessing.Normalization()
norm_layer.adapt(train_dataset.map(lambda x, y: tf.reshape(x, input_shape)))

# we have made an application and embeded wandb in the recording process
this is to allow us to :
> - capture new data from you device 🔊
> - Version data using wandb Atefact 🏺
> - Define your wakework 📢

In [140]:
!tree data -d


data
├── background
├── no
└── yes

3 directories


# Create a new training run for our training 🏃🏻‍♀️ ♀️ 

In [141]:
run = wandb.init(config={"bs": 12},entity='tiny-ml',project = 'wake_word_detection', group='training')

## Define our model 🏗️

In [142]:
baseline_model = tf.keras.models.Sequential([
  tf.keras.layers.Input(shape=input_shape),
  tf.keras.layers.experimental.preprocessing.Resizing(32, 32, interpolation="nearest"), 
  norm_layer,
  tf.keras.layers.Conv2D(8, kernel_size=(8,8), strides=(2, 2), activation="relu"),
  tf.keras.layers.MaxPool2D(pool_size=(2,2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Dense(50, activation='softmax')
])

In [143]:
baseline_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing_6 (Resizing)       (None, 32, 32, 1)         0         
                                                                 
 normalization_6 (Normalizat  (None, 32, 32, 1)        3         
 ion)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 13, 13, 8)         520       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 6, 6, 8)          0         
 2D)                                                             
                                                                 
 flatten_6 (Flatten)         (None, 288)               0         
                                                                 
 dropout_6 (Dropout)         (None, 288)              

In [144]:
METRICS = ["accuracy",]
baseline_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.003),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=METRICS,
)

In [145]:
def scheduler(epoch, lr):
    ''' a function to increase lr at start of trining
    '''
    if epoch < 10:
        return lr
    else:
        # add somthing like np.linespace([0,-0.1])
        return lr * tf.math.exp(-0.1)

In [146]:
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint
# Initialize a new W&B run 
checkpoint_path = "training_1/"
checkpoint_dir = os.path.dirname(checkpoint_path)
# Create a callback that saves the model's weights

In [147]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    verbose=1
)

In [148]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(verbose=0, patience=25), 
    tf.keras.callbacks.LearningRateScheduler(scheduler)
,cp_callback,WandbMetricsLogger(),WandbModelCheckpoint(checkpoint_path)]

In [149]:
run

In [150]:
EPOCHS = 250
history = baseline_model.fit(
     train_ds, 
    epochs=EPOCHS,
    callbacks=callbacks,
    validation_data=val_ds
)

Epoch 1/250
361/365 [============================>.] - ETA: 0s - loss: 3.4901 - accuracy: 0.1098
Epoch 1: saving model to training_1/


INFO:tensorflow:Assets written to: training_1/assets


INFO:tensorflow:Assets written to: training_1/assets


365/365 [==============================] - 5s 12ms/step - loss: 3.4873 - accuracy: 0.1103 - val_loss: 3.1569 - val_accuracy: 0.1898 - lr: 0.0030
Epoch 2/250
361/365 [============================>.] - ETA: 0s - loss: 3.1311 - accuracy: 0.1892
Epoch 2: saving model to training_1/


INFO:tensorflow:Assets written to: training_1/assets


...

INFO:tensorflow:Assets written to: training_1/assets


365/365 [==============================] - 2s 6ms/step - loss: 2.4324 - accuracy: 0.3437 - val_loss: 2.8650 - val_accuracy: 0.2583 - lr: 5.4947e-05
Epoch 51/250
359/365 [============================>.] - ETA: 0s - loss: 2.4274 - accuracy: 0.3453
Epoch 51: saving model to training_1/


INFO:tensorflow:Assets written to: training_1/assets


INFO:tensorflow:Assets written to: training_1/assets


365/365 [==============================] - 2s 6ms/step - loss: 2.4270 - accuracy: 0.3455 - val_loss: 2.8636 - val_accuracy: 0.2602 - lr: 4.9718e-05


In [151]:
run.finish()

### Evaluate our locally saved model

In [156]:
latest = tf.train.latest_checkpoint('training_1')
baseline_model.load_weights(latest)

2023-01-29 21:42:26.478528: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open training_1/.: FAILED_PRECONDITION: training_1; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


### Evaluate our Pre-trained Model

In [157]:
baseline_model.evaluate(test_ds, verbose=2)

214/214 - 5s - loss: 2.4582 - accuracy: 0.3518 - 5s/epoch - 23ms/step


[2.4581873416900635, 0.3517705500125885]

### Dowload model artifact/ our registered model  

In [160]:
api = wandb.Api()
artifact = api.artifact(f'tiny-ml/wake_word_detection/run_{run.id}_model:v50', type='model')
print(artifact.digest,artifact.aliases)
file = artifact.download()  

fb60627ca17c932d33d95875ef1c3b14 ['latest', 'epoch_50']


### Load our pre trained tiny model from wandb artifact

In [161]:
baseline_model.load_weights(f'{file}/cp.ckpt')
baseline_model.evaluate(test_ds, verbose=2)

214/214 - 2s - loss: 2.5362 - accuracy: 0.3197 - 2s/epoch - 8ms/step


[2.5361759662628174, 0.3197249174118042]

In [163]:
sample_rate = 16000
get_arm_spectrogram = Arm_spect().get_arm_spectrogram
path = Path('./data/') 
paths = {pth.name:list(map(str,filter(lambda x:x.name != '.gitkeep',pth.iterdir()))) 
         for pth in path.iterdir() if pth.name in ['background','yes']}
paths = {k:np.stack(list(filter(lambda x:len(x)==16384,map(read_wav,tqdm(v))))) 
         for (k,v) in tqdm(paths.items())}

for k,v in paths.items():
    filler_x = [augmenter(x,sample_rate) 
                for x in v[np.random.choice(v.shape[0], 500-len(v), replace=True)]]
    filler_x = np.stack(list(map(lambda x:x[:16384], tqdm(filler_x))))
    paths[k]=np.array(list(map(get_arm_spectrogram,tqdm(np.vstack([v,filler_x]))))).astype(np.float32)


100%|████████████████████████████████████████| 500/500 [00:01<00:00, 385.55it/s]


In [164]:
paths.values()

dict_values([array([[[1.07015625e+02, 6.92031250e+01, 1.55312500e+01, ...,
         1.71875000e-01, 1.56250000e-01, 1.56250000e-01],
        [1.22921875e+02, 6.72656250e+01, 4.59375000e+00, ...,
         6.25000000e-02, 6.25000000e-02, 7.81250000e-02],
        [1.27843750e+02, 6.41562500e+01, 1.71875000e-01, ...,
         0.00000000e+00, 0.00000000e+00, 1.56250000e-02],
        ...,
        [1.28031250e+02, 6.39843750e+01, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.28031250e+02, 6.39843750e+01, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.28031250e+02, 6.39843750e+01, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

       [[1.07015625e+02, 6.92031250e+01, 1.55312500e+01, ...,
         1.71875000e-01, 1.56250000e-01, 1.56250000e-01],
        [1.22921875e+02, 6.72656250e+01, 4.59375000e+00, ...,
         6.25000000e-02, 6.25000000e-02, 7.81250000e-02],
        [1.2

In [165]:
y = np.concatenate([np.zeros((1,500)).astype(np.uint16)[0], np.ones((1,500)).astype(np.uint16)[0]])
x = np.vstack(list(paths.values()))

In [167]:
train_wake_data = tf.data.Dataset.from_tensor_slices((tf.cast(x,tf.float32), y))
train_wake_loader = train_wake_data.cache().shuffle(2000, seed=42).batch(32).prefetch(tf.data.AUTOTUNE)

In [168]:
x,y = next(iter(train_wake_data.take(1)))
print(f' target = {y}, \n spectrogram = \n {x}')
input_shape = tf.expand_dims(x, axis=-1).shape
norm_layer = tf.keras.layers.experimental.preprocessing.Normalization()
norm_layer.adapt(train_wake_data.map(lambda x, y: tf.reshape(x, input_shape)))


 target = 0, 
 spectrogram = 
 [[1.07015625e+02 6.92031250e+01 1.55312500e+01 ... 1.71875000e-01
  1.56250000e-01 1.56250000e-01]
 [1.22921875e+02 6.72656250e+01 4.59375000e+00 ... 6.25000000e-02
  6.25000000e-02 7.81250000e-02]
 [1.27843750e+02 6.41562500e+01 1.71875000e-01 ... 0.00000000e+00
  0.00000000e+00 1.56250000e-02]
 ...
 [1.28031250e+02 6.39843750e+01 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.28031250e+02 6.39843750e+01 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.28031250e+02 6.39843750e+01 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


In [169]:
input_shape = tf.expand_dims(x, axis=-1).shape
print(input_shape)

(249, 257, 1)


In [170]:
base_model = tf.keras.models.Sequential([
  tf.keras.layers.Input(shape=input_shape),
  tf.keras.layers.experimental.preprocessing.Resizing(32, 32, interpolation="nearest"), 
  norm_layer,
  tf.keras.layers.Conv2D(8, kernel_size=(8,8), strides=(2, 2), activation="relu"),
  tf.keras.layers.MaxPool2D(pool_size=(2,2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Dense(50, activation='softmax')
])

In [171]:
base_model.load_weights('./training_1/')

In [172]:
model_body = tf.keras.Model(inputs=base_model.input, outputs=base_model.layers[-2].output)
classifier_head = tf.keras.layers.Dense(1, activation="sigmoid")(model_body.output)
wake_word_model = tf.keras.Model(model_body.input, classifier_head)
wake_word_model.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 249, 257, 1)]     0         
                                                                 
 resizing_7 (Resizing)       (None, 32, 32, 1)         0         
                                                                 
 normalization_7 (Normalizat  (None, 32, 32, 1)        3         
 ion)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 13, 13, 8)         520       
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 6, 6, 8)          0         
 2D)                                                             
                                                                 
 flatten_7 (Flatten)         (None, 288)               0   

In [173]:
for layer in wake_word_model.layers:
    layer.trainable = False
wake_word_model.layers[-1].trainable = True

In [174]:
# Initialize a new W&B run
run = wandb.init(config={"bs": 12},entity='tiny-ml',project = 'wake_word_detection', group='training',settings={'quiet':True,'silent':True,'show_warnings':False,'show_info':False})
checkpoint_path = "training_2/"
checkpoint_dir = os.path.dirname(checkpoint_path)
print(checkpoint_dir)
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    verbose=1
)
callbacks = [
    tf.keras.callbacks.EarlyStopping(verbose=0, patience=25), 
    tf.keras.callbacks.LearningRateScheduler(scheduler)
,cp_callback,WandbMetricsLogger(),WandbModelCheckpoint(checkpoint_path)]

training_2


In [175]:
wake_word_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.003),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=METRICS,
)

In [176]:
EPOCHS = 100
history = wake_word_model.fit(
    train_wake_loader, 
    epochs=EPOCHS,
    callbacks=callbacks
)

Epoch 1/100
32/32 [==============================] - ETA: 0s - loss: 0.6566 - accuracy: 0.6530WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy



Epoch 1: saving model to training_2/


INFO:tensorflow:Assets written to: training_2/assets


INFO:tensorflow:Assets written to: training_2/assets


32/32 [==============================] - 1s 26ms/step - loss: 0.6566 - accuracy: 0.6530 - lr: 0.0030
Epoch 2/100
23/32 [====================>.........] - ETA: 0s - loss: 0.5169 - accuracy: 0.7812WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy


...

32/32 [==============================] - 1s 27ms/step - loss: 0.3343 - accuracy: 0.8740 - lr: 4.0917e-07
Epoch 100/100
26/32 [=======================>......] - ETA: 0s - loss: 0.3474 - accuracy: 0.8690WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy



Epoch 100: saving model to training_2/


INFO:tensorflow:Assets written to: training_2/assets


INFO:tensorflow:Assets written to: training_2/assets


32/32 [==============================] - 1s 38ms/step - loss: 0.3422 - accuracy: 0.8690 - lr: 3.7023e-07


In [190]:
for layer in wake_word_model.layers:
    layer.trainable = True

In [191]:

import tensorflow_model_optimization as tfmot

def apply_qat_to_dense_and_cnn(layer):
  if isinstance(layer, (tf.keras.layers.Dense, tf.keras.layers.Conv2D)):
    return tfmot.quantization.keras.quantize_annotate_layer(layer)
  return layer

annotated_model = tf.keras.models.clone_model(
    wake_word_model,
    clone_function=apply_qat_to_dense_and_cnn,
)

quant_aware_model = tfmot.quantization.keras.quantize_apply(annotated_model)
quant_aware_model.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 249, 257, 1)]     0         
                                                                 
 resizing_7 (Resizing)       (None, 32, 32, 1)         0         
                                                                 
 normalization_7 (Normalizat  (None, 32, 32, 1)        3         
 ion)                                                            
                                                                 
 quant_conv2d_7 (QuantizeWra  (None, 13, 13, 8)        539       
 pperV2)                                                         
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 6, 6, 8)          0         
 2D)                                                             
                                                           

In [197]:
quant_aware_model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=METRICS,
)

EPOCHS=100
quant_aware_history = quant_aware_model.fit(
    train_wake_loader,  
    epochs=EPOCHS
)

Epoch 1/100
32/32 [==============================] - 1s 3ms/step - loss: 0.3295 - accuracy: 0.8700
Epoch 2/100
32/32 [==============================] - 0s 3ms/step - loss: 0.2932 - accuracy: 0.8800
Epoch 3/100
32/32 [==============================] - 0s 3ms/step - loss: 0.3004 - accuracy: 0.8810
Epoch 4/100
32/32 [==============================] - 0s 3ms/step - loss: 0.2875 - accuracy: 0.8880
Epoch 5/100
32/32 [==============================] - 0s 3ms/step - loss: 0.3177 - accuracy: 0.8870
Epoch 6/100
32/32 [==============================] - 0s 3ms/step - loss: 0.2913 - accuracy: 0.8890
Epoch 7/100
32/32 [==============================] - 0s 5ms/step - loss: 0.2545 - accuracy: 0.8830
Epoch 8/100
32/32 [==============================] - 0s 3ms/step - loss: 0.2657 - accuracy: 0.9040
Epoch 9/100
32/32 [==============================] - 0s 3ms/step - loss: 0.2729 - accuracy: 0.8990
Epoch 10/100
32/32 [==============================] - 0s 3ms/step - loss: 0.2448 - accuracy: 0.9070
Epoch 11/

32/32 [==============================] - 0s 3ms/step - loss: 0.1619 - accuracy: 0.9320
Epoch 84/100
32/32 [==============================] - 0s 3ms/step - loss: 0.1634 - accuracy: 0.9350
Epoch 85/100
32/32 [==============================] - 0s 3ms/step - loss: 0.1693 - accuracy: 0.9330
Epoch 86/100
32/32 [==============================] - 0s 3ms/step - loss: 0.1794 - accuracy: 0.9300
Epoch 87/100
32/32 [==============================] - 0s 3ms/step - loss: 0.1690 - accuracy: 0.9350
Epoch 88/100
32/32 [==============================] - 0s 3ms/step - loss: 0.1643 - accuracy: 0.9350
Epoch 89/100
32/32 [==============================] - 0s 3ms/step - loss: 0.1597 - accuracy: 0.9320
Epoch 90/100
32/32 [==============================] - 0s 3ms/step - loss: 0.1420 - accuracy: 0.9420
Epoch 91/100
32/32 [==============================] - 0s 3ms/step - loss: 0.1490 - accuracy: 0.9430
Epoch 92/100
32/32 [==============================] - 0s 3ms/step - loss: 0.1544 - accuracy: 0.9390
Epoch 93/100


In [198]:
converter = tf.lite.TFLiteConverter.from_keras_model(quant_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model_quant = converter.convert()

INFO:tensorflow:Assets written to: /var/folders/q8/9dhs6y4d21s87vxtnwqflq5m0000gn/T/tmp4_1be5sa/assets


INFO:tensorflow:Assets written to: /var/folders/q8/9dhs6y4d21s87vxtnwqflq5m0000gn/T/tmp4_1be5sa/assets
/Users/fridadesigley/.pyenv/versions/3.10.6/envs/tiny-ml/lib/python3.10/site-packages/tensorflow/lite/python/convert.py:765: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-01-29 22:08:23.786690: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-01-29 22:08:23.786702: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-01-29 22:08:23.786789: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/q8/9dhs6y4d21s87vxtnwqflq5m0000gn/T/tmp4_1be5sa
2023-01-29 22:08:23.788279: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-01-29 22:08:23.788285: I tensorflow/cc/saved_model/reader.cc:130] Readin

In [204]:
tflite_models_dir = Path("tf_lite_model")
tflite_models_dir.mkdir(exist_ok=True, parents=True)
tflite_model_file = tflite_models_dir/"wake_word.tflite"
tflite_model_file.write_bytes(tflite_model_quant)

6016

In [207]:
%%bash
echo "alignas(8) const unsigned char tflite_model[] = {" > tf_lite_model/wake_word.tflite.h
cat tf_lite_model/wake_word.tflite | xxd -i                        >> tf_lite_model/wake_word.tflite.h
echo "};"                                               >> tf_lite_model/wake_word.tflite.h

In [208]:
!ls tf_lite_model/

wake_word.tflite   wake_word.tflite.h
